In [16]:
import pandas as pd
from pandas import DataFrame
import datetime
import numpy as np
import statsmodels.api as sm
import math
import julian
from datetime import date
from dateutil.relativedelta import relativedelta

In [17]:
estacao = pd.read_csv('utils/lista_estacoes.csv', sep=',')

vazao_output = DataFrame(index=[x for x in range(len(estacao.id))], columns=[
                         'Estacao', 'Localizacao', 'Qsup', 'VazaoMinPrev', 'Status', 'isForecast'])

now = date(2021, 5, 1)

modeldates = [date(2021, 5, 1), date(2021, 6, 1), date(2021, 7, 1)]

In [24]:
estacao

,id,localizacao,alfa,q90,vsup
0,45590000,Rio Correntina em Correntina,-0.00124,23.2,1.0
1,45740001,Rio do Meio em Mocambo,-0.00216,24.8,9.0
2,45770000,Rio Arrojado em Arrojado,-0.00106,43.0,14.4
3,45840000,Rio Formoso em Gatos,-0.00115,54.4,17.2
4,45880000,Rio Formoso em Colônia do Formoso,-0.00116,55.4,17.4
5,45960001,Rio Corrente em Porto Novo,-0.00163,138.7,45.2
6,46415000,Rio Grande em Sítio Grande,-0.00239,19.4,13.2
7,46543000,Rio de Ondas em Fazenda Rendenção,-0.00236,32.5,9.3
8,46590000,Rio Branco em Nova Vida Montante,-0.00146,38.2,13.0
9,46675000,Rio Grande em Fazenda Macambira,-0.00296,132.5,48.4


In [18]:
if now in modeldates:
    a_month = relativedelta(months=1)
    start_date = (now - a_month).strftime('%d/%m/%Y')
    end_date = now.strftime('%d/%m/%Y')
else:
    print('Some error was occurred!')

In [19]:
d0 = date(now.year, 1, 1)

d2 = date(now.year, 10, 31)

a_month = relativedelta(months=1)

dt1 = now - relativedelta(days=1)
dt = dt1 - d0
dt2 = d2 - dt1

mes_prev = now.month - 1

In [27]:
for est in range(len(estacao.id)):
 # ler o arquivo e mostrar a separação
    dados_telemetria = pd.read_csv(
        'filtrado/'+str(estacao.id[est])+'_filtrado.csv', sep=';', encoding='latin-1')
    dados_telemetria = pd.DataFrame(dados_telemetria)
    dados_telemetria['data'] = pd.to_datetime(dados_telemetria['data'])
    dados_telemetria['dia'] = dados_telemetria['data'].dt.dayofyear
    dados_telemetria['mes'] = dados_telemetria['data'].dt.month
    dados_telemetria.drop(
        dados_telemetria.loc[dados_telemetria['vazao diaria'] == 0].index, inplace=True)

    alfa_pond = estacao.alfa[est]
    q_noventa = estacao.q90[est]
    vsup = estacao.vsup[est]

    dados_mes_prev = dados_telemetria.loc[(
        dados_telemetria['mes'] == mes_prev)]
    dados_mes_prev.drop(
        dados_mes_prev.loc[dados_mes_prev['vazao diaria'] == 0].index, inplace=True)
    dados_mes_prev.drop(dados_mes_prev.loc[dados_mes_prev['vazao diaria']
                                           != dados_mes_prev['vazao diaria'].min()].index, inplace=True)
    dados_mes_prev.drop(
        dados_mes_prev.loc[dados_mes_prev['dia'] != dados_mes_prev['dia'].max()].index, inplace=True)

    dia_valor = np.array(dados_mes_prev['dia'])

    if not dia_valor:
        vazao_output['Estacao'][est] = str(estacao.id[est])
        vazao_output['Localizacao'][est] = str(estacao.localizacao[est])
        vazao_output['Qsup'][est] = str(estacao.vsup[est])
        vazao_output['VazaoMinPrev'][est] = 0.0
        vazao_output['Status'][est] = 'Sem previsão'
        vazao_output['isForecast'][est] = 'no'
        continue

        quantidade_dias = dt.days - int(dia_valor) + dt2.days

        dia = {}
        b = 0
        for i in range(quantidade_dias):
            dia[str(b)] = dia_valor + i
            b = b+1

        dia = pd.DataFrame.from_dict(dia, orient='index', columns=['dia_do_ano'])

        qo = np.array(dados_mes_prev['vazao diaria'])
        to = np.array(dados_mes_prev['dia'])
        valor_mask = to[0]
        dado_selecionado_mask = dados_mes_prev.loc[(
            dados_mes_prev['mes'] == mes_prev)]

        dado_selecionado_mask = dado_selecionado_mask.mask(
            dado_selecionado_mask['dia'] < valor_mask)
        dado_final = pd.concat([dia, dado_selecionado_mask])

        dia_periodo = dia['dia_do_ano']
        vazao_calculada = {}

        k = 0
        for dias in range(len(dia_periodo)):
            vazao_calculada[str(k)] = (
                qo[0]*(np.exp((alfa_pond)*(dia_periodo[dias]-to[0]))))
            k = k+1

        vazao_feita = pd.DataFrame.from_dict(
            vazao_calculada, orient='index', columns=['vazao'])
        vazao_minima_prevista = vazao_feita["vazao"].iloc[-1]

        if(vsup <= 0.70*vazao_minima_prevista):
            status = 'Baixo'
        elif (vsup > 0.70*vazao_minima_prevista and vsup <= vazao_minima_prevista):
            status = 'Moderado'
        else:
            status = "Alto"

        vazao_output['Estacao'][est] = str(estacao.id[est])
        vazao_output['Localizacao'][est] = str(estacao.localizacao[est])
        vazao_output['Qsup'][est] = str(estacao.vsup[est])
        vazao_output['VazaoMinPrev'][est] = vazao_minima_prevista
        vazao_output['Status'][est] = status
        vazao_output['isForecast'][est] = 'yes'

    vazao_output.to_csv('hidroforecast.csv', sep=',')

ipykernel_launcher:54: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

ipykernel_launcher:27: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
